<!-- SUMMARY: Check the Inheritance technique in the Python lnaguage -->

<!-- CATEGORY: Tools_for_Python_environment -->

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp

In [ ]:
db = gl.DbGrid.create([50,50],[2,2])
mesh = gl.MeshETurbo([100,100])
proj = gl.ProjMatrix.create(db,mesh)

In [ ]:
A = gl.TestInheritance()
A.setIproj(proj)

In [ ]:
class SpecializedProj(gl.IProj):
    def __init__(self,n):
        super(SpecializedProj,self).__init__()
        self.n = n
    def mesh2point(self,inv,outv):
        for i in range(inv.size):
            outv[i] = inv[i]
    def point2mesh(self,outv,inv):
        for i in range(inv.size):
            outv[i] = inv[i]
    def getNApex(self):
        return self.n
    def getNPoint(self):
        return self.n

In [ ]:
newproj = SpecializedProj(12)

In [ ]:
A.setIproj(newproj)

In [ ]:
A.display()

### Inheritance of an ICloneable class.

Add a clone method as follows:

In [ ]:
import numpy as np
class MyCov(gl.ACov):
    def __init__(self,range):
        super(MyCov,self).__init__()
        self.range = np.copy(range)
        #space is needed to compute the distance in _eval
        self.space = gl.getDefaultSpace() 
    #Specify the number of variables
    def getNVar(self):
        return 1
    #clone method
    def clone(self):
        return MyCov(self.range).__disown__()
    # _eval method (which is pure virtual in ACov)
    def _eval(self,p1,p2,ivar,jvar,mode):
        dist = self.space.getDistance(p1,p2)
        return np.exp(- dist/self.range)


In [ ]:
cov = MyCov(0.2)
p1 = gl.SpacePoint([1.,1.])
p2 = gl.SpacePoint([.95,.95])

print(cov.isNoStat())
cov.evalCov(p1,p2)

In [ ]:
db = gl.Db.createFillRandom(5,2,1)
grid = gl.DbGrid.create([100,100],[0.01,0.01])

model = gl.ModelGeneric()
model.setCov(cov)

modelgst = gl.Model.createFromParam(gl.ECov.EXPONENTIAL,range = cov.range,flagRange=False)

neigh = gl.NeighUnique()

In [ ]:
gl.kriging(db,grid,model,neigh)
gl.kriging(db,grid,modelgst,neigh)

Comparison of kriging with custom class and gstlearn class

In [ ]:
print(np.round(np.max(np.abs(grid["*estim"] - grid["*estim.1"])),5))